# <center> Table Of Content </center>
### [1. Introdactions](#intro) ###
### [1. About The Data](#about) ###
### [1. Importing Libraries](#im) ###
### [2. HyperParameters](#hp) ###
### [3. Data Loading and Preprocessing](#data) ###
### [4. DenseNet121](#den)  ###
### [5. Long Short-Term Memory](#lstm)
### [6. ResNet](#rf) ###

<hr>

<a id="intro"></a>
# <center> Introdactions </center>

In this notebook, you will  compare three classification algorithms.


Below, is where we are going to use the classification algorithms to create a model based on our training data and evaluate our testing data.

We will use some of the algorithms taught in the course, specifically:

1. Convolutional Neural Network (DenseNet121)
2. Recurrent neural network (Long Short-Term Memory)
3. Faster R_CNN (ResNet50)

We will evaluate our models using:

1.  Accuracy Score
2.  F1-Score
3.  Recall Score
4.  Precision Score 

Finally, you will use your models to generate the report at the end. 

<a id="intro"></a>
# <center> About The Data </center>

UCF crime dataset used for Real-world Anomaly Detection in Surveillance Videos.

The dataset contains images extracted from every video from the UCF Crime Dataset.
Every 10th frame is extracted from each full-length video and combined for every video in that class.
All the images are of size 64*64 and in (.png) format

The dataset has a total of 14 Classes :

1. Abuse 
2. Arrest'
3. Arson
4. Assault
5. Burglary
6. Explosion
7. Fighting
8. Normal Videos
9. RoadAccidents
10. Robbery
11. Shooting
12. Shoplifting
13. Stealing
14. Vandalism

The total image count for the train subset is 1,266,345.
The total image count for the test subset is 111,308.

Acknowledgements
All videos used for frame extraction were obtained from UCF CRIME OFFICIAL DATASET.
Official Dropbox link for all videos can be found [here](https://www.dropbox.com/sh/75v5ehq4cdg5g5g/AABvnJSwZI7zXb8_myBA0CLHa?dl=0).

<a id="im"></a>
# <center>IMPORTING LIBRARIES</center> 

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import roc_curve, auc, roc_auc_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

<a id="hp"></a>
# <center>HYPERPARAMETRERS AND DIRECTORIES</center>

In [13]:
train_dir = "../input/ucf-crime-dataset/Train"
test_dir = "../input/ucf-crime-dataset/Test"

SEED = 12
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 64
EPOCHS = 1
LR =  0.00003
NUM_CLASSES = 14
CLASS_LABELS = ['Abuse','Arrest','Arson','Assault','Burglary','Explosion','Fighting',"Normal",'RoadAccidents','Robbery','Shooting','Shoplifting','Stealing','Vandalism']

<a id="data"></a>
# <center> DATA PRE-PROCESSING</center>

Creating two different generator for train and test data

In [14]:
preprocess_fun = tf.keras.applications.densenet.preprocess_input

train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.05,
                                   rescale = 1./255,
                                   preprocessing_function=preprocess_fun
                                  )
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  preprocessing_function=preprocess_fun
                                 )

In [15]:
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = True , 
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    seed = SEED
                                                   )
test_generator = test_datagen.flow_from_directory(directory = test_dir,
                                                   target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = False , 
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    seed = SEED
                                                  )

Found 1266345 images belonging to 14 classes.
Found 111308 images belonging to 14 classes.


## Data distribution (count) among all the classes

### Train data Distribution

In [16]:
fig = px.bar(x = CLASS_LABELS,
             y = [list(train_generator.classes).count(i) for i in np.unique(train_generator.classes)] , 
             color = np.unique(train_generator.classes) ,
             color_continuous_scale="Emrld") 
fig.update_xaxes(title="Classes")
fig.update_yaxes(title = "Number of Images")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Train Data Distribution ',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

### Test Data Distribution

In [17]:
fig = px.bar(x = CLASS_LABELS,
             y = [list(test_generator.classes).count(i) for i in np.unique(test_generator.classes)] , 
             color = np.unique(train_generator.classes) ,
             color_continuous_scale="Emrld") 
fig.update_xaxes(title="Classes")
fig.update_yaxes(title = "Number of Images")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Test Data Distribution ',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

<a id="den"></a>
# <center> DenseNet121 </center>

In [18]:
def feature_extractor(inputs):
    feature_extractor = tf.keras.applications.DenseNet121(input_shape=(IMG_HEIGHT,IMG_WIDTH, 3),
                                               include_top=False,
                                               weights="imagenet")(inputs)
    
    return feature_extractor

def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.4) (x)
    x = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax", name="classification")(x)
    
    return x

def final_model(inputs):
    densenet_feature_extractor = feature_extractor(inputs)
    classification_output = classifier(densenet_feature_extractor)
    
    return classification_output

def define_compile_model():
    
    inputs = tf.keras.layers.Input(shape=(IMG_HEIGHT ,IMG_WIDTH,3))
    classification_output = final_model(inputs) 
    model = tf.keras.Model(inputs=inputs, outputs = classification_output)
     
    model.compile(optimizer=tf.keras.optimizers.SGD(LR), 
                loss='categorical_crossentropy',
                metrics = [tf.keras.metrics.AUC()])
  
    return model

DenseNet121_model = define_compile_model()
clear_output()
DenseNet121_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
densenet121 (Functional)     (None, 2, 2, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              263168    
_________________________________________________________________
dropout_7 (Dropout)          (None, 1024)              0   

<a id="train"></a>
### Training DenseNet121 

In [ ]:
denNet_history = DenseNet121_model.fit(x = train_generator,validation_data=test_generator,epochs = EPOCHS)

19395/19787 [============================>.] - ETA: 2:03 - loss: 1.2394 - auc_2: 0.9081

<a id="lstm"></a>
# <center> Long Short-Term Memory </center>

In [ ]:
def define_compile_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(256, return_sequences=True, input_shape=(timesteps, input_dim)))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.LSTM(128))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.SGD(LR),
                  loss='categorical_crossentropy',
                  metrics=[tf.keras.metrics.AUC()])

    return model

lstm_model = define_compile_model()
clear_output()
lstm_model.summary()

### Training LSTM

In [ ]:
lstm_history = lstm_model.fit(x = train_generator,validation_data=test_generator,epochs = EPOCHS)

<a id='rf'><a/>
# <center> ResNet50 </center>

In [ ]:
def feature_extractor(inputs):
    feature_extractor = ResNet50(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                                 include_top=False,
                                 weights="imagenet")(inputs)
    return feature_extractor

def classifier(inputs):
    x = GlobalAveragePooling2D()(inputs)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation="relu")(x)
    x = Dropout(0.4)(x)
    x = Dense(NUM_CLASSES, activation="softmax", name="classification")(x)
    return x

def final_model(inputs):
    resnet_feature_extractor = feature_extractor(inputs)
    classification_output = classifier(resnet_feature_extractor)
    return classification_output

def define_compile_model():
    inputs = tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    classification_output = final_model(inputs)
    model = Model(inputs=inputs, outputs=classification_output)
     
    model.compile(optimizer=tf.keras.optimizers.SGD(LR), 
                  loss='categorical_crossentropy',
                  metrics=[tf.keras.metrics.AUC()])
  
    return model

reaNet_model = define_compile_model()
reaNet_model.summary()

### Traing ResNet50

In [ ]:
resNet_history = reaNet_model.fit(x = train_generator,validation_data=test_generator,epochs = EPOCHS)

# <center>Save Models</center>

In [ ]:
DenseNet121_model.save("densenet121_model")
lstm_model.save("lstm_model")
reaNet_model.save("resnet50_model")

<a id="auc"></a>
# <center> Multiclass AUC Curve </center> 

In [ ]:
preds = model.predict(test_generator)
y_test = test_generator.classes
fig, c_ax = plt.subplots(1,1, figsize = (15,8))

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    for (idx, c_label) in enumerate(CLASS_LABELS):
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr,lw=2, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'black',linestyle='dashed', lw=4, label = 'Random Guessing')
    return roc_auc_score(y_test, y_pred, average=average)

print('ROC AUC score:', multiclass_roc_auc_score(y_test , preds  , average = "micro"))
plt.xlabel('FALSE POSITIVE RATE', fontsize=18)
plt.ylabel('TRUE POSITIVE RATE', fontsize=16)
plt.legend(fontsize = 11.5)
plt.show()